In [1]:
# Importando los módulos que necesitaremos
import numpy as np

import pandas as pd
from pandas.plotting import table

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

import seaborn as sns

from datetime import datetime

import os

# Configurando los estílos de los gráficos
plt.ioff()
sns.set_context('talk')
sns.set_style("whitegrid")

# Definiendo Constantes
BINS = [0, 10, 20, 30, 40, 50, 60, 70, 80, 300]

## Limpieza de Datos

In [2]:
# Leo la fuente de datos local en CSV 
df = pd.read_csv('data/vacunation_data.csv')
df.head()

,FECHA_CORTE,UUID,GRUPO_RIESGO,EDAD,SEXO,FECHA_VACUNACION,DOSIS,FABRICANTE,DIRESA,DEPARTAMENTO,PROVINCIA,DISTRITO
0,20210402,c7ba6f9424c59a543e8803495c1061db,PERSONAL DE SALUD,39.0,MASCULINO,20210225,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,CHICLAYO
1,20210402,c7ba6f9424c59a543e8803495c1061db,PERSONAL DE SALUD,39.0,MASCULINO,20210318,2,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,CHICLAYO
2,20210402,51ffbe83478f1e64da17fe2756a9594b,PERSONAL DE SALUD,37.0,FEMENINO,20210219,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,JOSE LEONARDO ORTIZ
3,20210402,51ffbe83478f1e64da17fe2756a9594b,PERSONAL DE SALUD,37.0,FEMENINO,20210312,2,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,JOSE LEONARDO ORTIZ
4,20210402,54048fe3b06c35f14350d7e81340bdbd,PERSONAL DE SALUD,44.0,MASCULINO,20210220,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,JOSE LEONARDO ORTIZ


In [3]:
# Obtengo el conteo del número de valores en cada columnas
total_count = len(df)
print(f'TOTAL COUNT: {total_count}')
print(df.count())

TOTAL COUNT: 857493
FECHA_CORTE         857493
UUID                857493
GRUPO_RIESGO        857493
EDAD                799142
SEXO                857493
FECHA_VACUNACION    857493
DOSIS               857493
FABRICANTE          857493
DIRESA              857493
DEPARTAMENTO        857493
PROVINCIA           857493
DISTRITO            857493
dtype: int64


In [4]:
# Verificamos que las columnas DEPARTAMENTO, GRUPO_RIESGO y SEXO tengan los valores indicados
print(df['DEPARTAMENTO'].unique())
print(df['GRUPO_RIESGO'].unique())
print(df['SEXO'].unique())

['LAMBAYEQUE' 'LIMA' 'AMAZONAS' 'CAJAMARCA' 'LA LIBERTAD' 'UCAYALI'
 'AREQUIPA' 'AYACUCHO' 'APURIMAC' 'SAN MARTIN' 'PIURA' 'HUANUCO'
 'MADRE DE DIOS' 'LORETO' 'CALLAO' 'CUSCO' 'ICA' 'HUANCAVELICA' 'JUNIN'
 'PUNO' 'PASCO' 'ANCASH' 'MOQUEGUA' 'TACNA' 'TUMBES']
['PERSONAL DE SALUD' 'TRABAJADOR Ó PERSONAL DE LIMPIEZA'
 'PERSONAL MILITAR Ó FF AA' 'POLICIA NACIONAL DEL PERU' 'ADULTO MAYOR'
 'PERSONAL DE SEGURIDAD' 'ESTUDIANTES DE CIENCIAS DE LA SALUD'
 'BRIGADISTAS']
['MASCULINO' 'FEMENINO']


In [5]:
# Filtramos las filas que no tienen valor en las columnas EDAD o SEXO
df = df[df['EDAD'].notnull() & df['SEXO'].notnull()]

## Transformación de Datos

In [6]:
# Abreviando los grupos de riesgo
df['GRUPO_RIESGO'].replace('PERSONAL DE SALUD','P. SALUD', inplace=True)
df['GRUPO_RIESGO'].replace('TRABAJADOR Ó PERSONAL DE LIMPIEZA','T. LIMP.', inplace=True)
df['GRUPO_RIESGO'].replace('PERSONAL DE SEGURIDAD','P. SEG.', inplace=True)
df['GRUPO_RIESGO'].replace('PERSONAL MILITAR Ó FF AA','FF.AA.', inplace=True)
df['GRUPO_RIESGO'].replace('POLICIA NACIONAL DEL PERU','P.N.P.', inplace=True)
df['GRUPO_RIESGO'].replace('ESTUDIANTES DE CIENCIAS DE LA SALUD','E. SALUD', inplace=True)
df['GRUPO_RIESGO'].replace('BRIGADISTAS','BRIG.', inplace=True)

# Genero Serie de grupos de riesgo
grupos_riesgo = df['GRUPO_RIESGO'].unique()
grupos_riesgo_series = pd.Series(np.zeros_like(grupos_riesgo),grupos_riesgo)

# Agrupo el conjunto de datos por DEPARTAMENTO, SEXO y EDAD para el DataFrame
gb_age = df[["DEPARTAMENTO","SEXO",'EDAD']].groupby(['DEPARTAMENTO', 'SEXO', pd.cut(df['EDAD'], BINS, False)])

# Agrupo el conjunto de datos por DEPARTAMENTO, SEXO y GRUPO_RIESGO para el DataFrame
gb_gr = df[["DEPARTAMENTO","SEXO",'GRUPO_RIESGO']].groupby(['DEPARTAMENTO', 'SEXO', 'GRUPO_RIESGO'])

In [7]:
def get_data_to_plot(department=None):
    if(department is None):
        return df[['SEXO','EDAD','GRUPO_RIESGO','FECHA_VACUNACION']]
    else:
        return df[df['DEPARTAMENTO'] == department][['SEXO','EDAD','GRUPO_RIESGO','FECHA_VACUNACION']]

## Visualización de Datos

In [8]:
# Visualizo los datos agrupados por EDAD via DataFrame
gb_age.size().unstack()

EDAD                     [0, 10)  [10, 20)  [20, 30)  [30, 40)  [40, 50)  \
DEPARTAMENTO  SEXO                                                         
AMAZONAS      FEMENINO         0        21      1396      2346      1369   
              MASCULINO        0        16       726      1314       944   
ANCASH        FEMENINO         0         3      3022      4783      3304   
              MASCULINO        0        36      2810      3184      1974   
APURIMAC      FEMENINO         0         4      1826      3160      2326   
              MASCULINO        0         6       709      1562      1481   
AREQUIPA      FEMENINO         0        56      4230      7598      5690   
              MASCULINO        0       579      3751      4490      3591   
AYACUCHO      FEMENINO         0        15      2001      3538      2588   
              MASCULINO        0        39       878      1785      1716   
CAJAMARCA     FEMENINO         0         6      3409      5346      3397   
              MASCULINO        0        23      1515      2999      2105   
CALLAO        FEMENINO         0       299      3978      6783      5403   
              MASCULINO        0      1035      9471      6850      6748   
CUSCO         FEMENINO         0        21      2329      5570      3756   
              MASCULINO        0        22      1019      2388      2170   
HUANCAVELICA  FEMENINO         0        11      1826      2625      1483   
              MASCULINO        0         8       683      1260       987   
HUANUCO       FEMENINO         0        10      2303      3823      2432   
              MASCULINO        0        24       955      2126      1563   
ICA           FEMENINO         0        17      2773      3791      2827   
              MASCULINO        0        17      1281      1921      1453   
JUNIN         FEMENINO         0         9      4370      6485      3988   
              MASCULINO        0        20      1433      2883      2129   
LA LIBERTAD   FEMENINO         0        11      4505      7495      4995   
              MASCULINO        0        46      3331      4159      3083   
LAMBAYEQUE    FEMENINO         0        14      2796      4942      3975   
              MASCULINO        0        16      1188      2411      2299   
LIMA          FEMENINO         0       370     25448     50827     38887   
              MASCULINO        0       964     24328     29793     25415   
LORETO        FEMENINO         0        16      1411      3098      2138   
              MASCULINO        0        47      1260      2253      1627   
MADRE DE DIOS FEMENINO         0         8       408       836       602   
              MASCULINO        0         2       186       457       347   
MOQUEGUA      FEMENINO         0         9      1003      1598      1334   
              MASCULINO        0         9       322       743       613   
PASCO         FEMENINO         0         6       966      1324       987   
              MASCULINO        0         7       358       667       637   
PIURA         FEMENINO         0        34      3382      5790      4989   
              MASCULINO        0        44      1689      2954      2669   
PUNO          FEMENINO         0        10      2079      4007      2726   
              MASCULINO        0         2       738      1813      1614   
SAN MARTIN    FEMENINO         0        64      2430      4088      2036   
              MASCULINO        0        47      1411      2601      1520   
TACNA         FEMENINO         0         7       992      1755      1622   
              MASCULINO        0        14       511       815       769   
TUMBES        FEMENINO         0         2       785      1173       981   
              MASCULINO        0         2       387       626       577   
UCAYALI       FEMENINO         0        21      1344      1982      1412   
              MASCULINO        0        62       974      1254       997   

EDAD                     [50, 60)  [60, 70)  [7

In [9]:
# Visualizo los datos agrupados por GRUPO_RIESGO via DataFrame
gb_gr.size().unstack()

GRUPO_RIESGO             ADULTO MAYOR  BRIG.  E. SALUD   FF.AA.  P. SALUD  \
DEPARTAMENTO  SEXO                                                          
AMAZONAS      FEMENINO            NaN    NaN       NaN     20.0    5830.0   
              MASCULINO           NaN    NaN       NaN     36.0    3570.0   
ANCASH        FEMENINO            NaN    NaN      82.0      NaN   13862.0   
              MASCULINO           NaN    NaN      52.0     10.0    6862.0   
APURIMAC      FEMENINO            NaN    NaN       NaN      3.0    8648.0   
              MASCULINO           NaN    NaN       4.0     14.0    4919.0   
AREQUIPA      FEMENINO         2035.0    NaN       3.0    435.0   24063.0   
              MASCULINO        1483.0    NaN       2.0   2994.0   10282.0   
AYACUCHO      FEMENINO            NaN    1.0       9.0      NaN    9956.0   
              MASCULINO           NaN    2.0       5.0    226.0    5770.0   
CAJAMARCA     FEMENINO            NaN    NaN      39.0      NaN   14145.0   
              MASCULINO           NaN    NaN      28.0      NaN    8035.0   
CALLAO        FEMENINO         2554.0    7.0      27.0   2734.0   19102.0   
              MASCULINO        1817.0    8.0      73.0  15078.0   10851.0   
CUSCO         FEMENINO          334.0    NaN      29.0      1.0   15408.0   
              MASCULINO         204.0    NaN      14.0      8.0    7764.0   
HUANCAVELICA  FEMENINO            NaN    NaN       NaN      NaN    6813.0   
              MASCULINO           NaN    NaN       NaN      7.0    3615.0   
HUANUCO       FEMENINO            NaN    NaN       8.0      NaN   10239.0   
              MASCULINO           NaN    NaN       5.0      NaN    5826.0   
ICA           FEMENINO            NaN    NaN      66.0     58.0   12880.0   
              MASCULINO           NaN    NaN      25.0    130.0    6262.0   
JUNIN         FEMENINO         1006.0    NaN     319.0     86.0   17911.0   
              MASCULINO         658.0    NaN      86.0    100.0    8337.0   
LA LIBERTAD   FEMENINO         3184.0   13.0      52.0      NaN   21957.0   
              MASCULINO        2459.0   11.0      22.0      NaN   11159.0   
LAMBAYEQUE    FEMENINO            1.0    NaN       6.0     97.0   16017.0   
              MASCULINO           NaN    NaN       NaN     26.0    8740.0   
LIMA          FEMENINO         8765.0   15.0     145.0   1431.0  152828.0   
              MASCULINO        6740.0   12.0      93.0   6609.0   80169.0   
LORETO        FEMENINO         2935.0   16.0      16.0     24.0    8256.0   
              MASCULINO        2716.0   13.0       3.0    181.0    6489.0   
MADRE DE DIOS FEMENINO            NaN    NaN       NaN     10.0    2298.0   
              MASCULINO           1.0    NaN       NaN     25.0    1360.0   
MOQUEGUA      FEMENINO            NaN    NaN       3.0      2.0    5026.0   
              MASCULINO           NaN    NaN       1.0     11.0    2427.0   
PASCO         FEMENINO            NaN    NaN       5.0      3.0    3996.0   
              MASCULINO           NaN    NaN       2.0      3.0    2182.0   
PIURA         FEMENINO            2.0    NaN      19.0    101.0   17782.0   
              MASCULINO           4.0    2.0      27.0    271.0    9943.0   
PUNO          FEMENINO            NaN    NaN      14.0      6.0   11649.0   
              MASCULINO           NaN    NaN       5.0     26.0    6474.0   
SAN MARTIN    FEMENINO         2601.0    NaN       NaN      NaN   10400.0   
              MASCULINO        2068.0    NaN       NaN      2.0    7066.0   
TACNA         FEMENINO            NaN    NaN      17.0     12.0    6146.0   
              MASCULINO           NaN    NaN       8.0     12.0    3300.0   
TUMBES        FEMENINO            NaN    NaN      40.0      NaN    3292.0   
              MASCULINO           NaN    NaN       4.0      NaN    1887.0   
UCAYALI       FEMENINO            1.0    NaN       NaN     14.0    5773.0   
              MASCULINO           NaN    NaN       NaN    228.0    3963.0

In [10]:
yticks = [5,15,25,35,45,55,65,75,85]

def generate_plot(area, data, date_range, show_plot=False, save_plot=True):

    # Creamos la figura, los ejes y agregamos la atribución
    plt.clf()

    fig, axs = plt.subplots(2,2, figsize=(20,20))
    plt.figtext(0.1,0.95, area + ': DISTRIBUCIÓN DE VACUNADOS CONTRA COVID-19 (N=' + str(len(data)) + ')', ha='left', fontsize=24)
    plt.figtext(0.1,0.925, 'Rango de fechas de vacunación: del {} al {}'.format(f'{date_range[0]:%Y-%m-%d}',f'{date_range[1]:%Y-%m-%d}'), ha='left', fontsize=20, color='#999')
    plt.figtext(0.9,0.02, """Fuente: https://www.datosabiertos.gob.pe/dataset/vacunaci%C3%B3n-contra-covid-19-ministerio-de-salud-minsa
    https://malexandersalazar.github.io/, Moisés Alexander Salazar Vila, """ + f'{datetime.now():%Y-%m-%d}', ha='right')

    # 1. Graficamos distribución por GRUPO ETAREO
    xlim_max = 0

    ## 1.1. Graficamos el primer histograma
    N, bins, patches = axs[0][0].hist(data["EDAD"], BINS, orientation = 'horizontal')

    ### Colorearemos las barras en base el conteo relativo de cada agrupación
    xlim_max = N.max() * 1.1
    fracs = N / N.max()
    norm = colors.Normalize(fracs.min(), fracs.max())

    ### Recorremos las barras y actualizamos cada color respectivamente
    for thisfrac, thispatch in zip(fracs, patches):
        color = plt.cm.viridis(norm(thisfrac))
        thispatch.set_facecolor(color)

    ### Personalizamos las ejes de nuestro gráfico
    axs[0][0].set_yticklabels([])
    axs[0][0].yaxis.set_tick_params(width=0)
    axs[0][0].set_yticks(yticks, minor=True)
    axs[0][0].set_yticklabels(['0-9','10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+'], minor=True)
    axs[0][0].set_ylim(0,90)
    axs[0][0].set_xlim(0,xlim_max)
    axs[0][0].set_ylabel('GRUPO ETÁREO (AÑOS)')

    ### Calculamos y anotamos los porcentajes correspondientes a cada barra
    for i in range(len(BINS)-1):
        axs[0][0].text(N[i], yticks[i]-1, str(round(N[i]*100/len(data),2)) + '%')

    ## 1.2. Graficamos el segundo histograma
    males = data[data['SEXO'] == 'MASCULINO']["EDAD"]
    females = data[data['SEXO'] == 'FEMENINO']["EDAD"]

    N, bins, patches = axs[0][1].hist([males, females], BINS, orientation = 'horizontal')
    
    ### Personalizamos las ejes de nuestro gráfico
    axs[0][1].set_yticklabels([])
    axs[0][1].yaxis.set_tick_params(width=0)
    axs[0][1].set_yticks(yticks, minor=True)
    axs[0][1].set_ylim(0,90)
    axs[0][1].set_xlim(0,xlim_max)

    ### Calculamos y anotamos los porcentajes correspondientes a cada barra
    for i in range(len(BINS)-1):
        axs[0][1].text(N[0][i], yticks[i]-3, str(round(N[0][i]*100/len(data),2)) + '%')
        axs[0][1].text(N[1][i], yticks[i]+1, str(round(N[1][i]*100/len(data),2)) + '%')

    axs[0][1].legend(['Hombres (n=' + str(len(males)) + ')', 'Mujeres (n=' + str(len(females)) + ')'])

    # 2. Graficamos distribución por GRUPO RIESGO
    xlim_max = 0

    ## 2.1. Graficamos el primer gráfico de barras 
    group_counts = data['GRUPO_RIESGO'].value_counts().add(grupos_riesgo_series,fill_value=0)
    group_counts.sort_index(inplace=True,ascending=False)
    pos = np.array(range(len(group_counts)))
    patches = axs[1][0].barh(pos, group_counts.values, align='center')
    
    ### Colorearemos las barras en base el conteo relativo de cada agrupación
    xlim_max = group_counts.values.max() * 1.1
    fracs = group_counts.values / group_counts.values.max()
    norm = colors.Normalize(fracs.min(), fracs.max())

    ### Recorremos las barras y actualizamos cada color respectivamente
    for thisfrac, thispatch in zip(fracs, patches):
        color = plt.cm.viridis(norm(thisfrac))
        thispatch.set_facecolor(color)

    ### Personalizamos las ejes de nuestro gráfico
    axs[1][0].set_yticks(pos)
    axs[1][0].set_yticklabels(list(group_counts.index.values))
    axs[1][0].set_xlim(0,xlim_max)

    ### Calculamos y anotamos los porcentajes correspondientes a cada barra
    for i in range(len(pos)):
        axs[1][0].text(group_counts.values[i], pos[i]-0.1, str(round(group_counts[i]*100/len(data),2)) + '%')

    ## 2.2. Graficamos el segundo gráfico de barras 
    males = data[data['SEXO'] == 'MASCULINO']
    males_group_counts = males["GRUPO_RIESGO"].value_counts().add(grupos_riesgo_series,fill_value=0)
    males_group_counts.sort_index(inplace=True,ascending=False)

    females = data[data['SEXO'] == 'FEMENINO']
    females_group_counts = females["GRUPO_RIESGO"].value_counts().add(grupos_riesgo_series,fill_value=0)
    females_group_counts.sort_index(inplace=True,ascending=False)

    pos = np.array(range(len(males_group_counts)))

    axs[1][1].barh(pos, males_group_counts.values, 0.35, align='center')
    axs[1][1].barh(pos+0.35, females_group_counts.values, 0.35, align='center')
    
    ### Personalizamos las ejes de nuestro gráfico
    axs[1][1].set_yticks(pos)
    axs[1][1].set_yticklabels([])
    axs[1][1].set_xlim(0,xlim_max)
    
    ### Calculamos y anotamos los porcentajes correspondientes a cada barra
    for i in range(len(pos)):
        axs[1][1].text(males_group_counts.values[i], pos[i]-0.1, str(round(males_group_counts[i]*100/len(data),2)) + '%')
        axs[1][1].text(females_group_counts.values[i], pos[i]+0.3, str(round(females_group_counts[i]*100/len(data),2)) + '%')

    ## 3. Mostramos, guardamos y generamos el Markdown para las imágenes

    sns.despine(left=False, bottom=False)

    if save_plot:
        filename = 'dist/{}_{}.png'.format(f'{datetime.now():%Y%m%d}', area.replace(' ', '_'))
        plt.savefig("../" + filename, bbox_inches='tight')
        with open("../dist/images.txt", "a", encoding='utf-8') as f:
            f.write('![alt text]({} "{}")\n\n'.format(filename, area))

    if show_plot:
        plt.show()

In [11]:
# Eliminando archivo generado para el Markdown de las imágenes
if os.path.exists("../dist/images.txt"):
    os.remove("../dist/images.txt")

In [12]:
# Perú
data = get_data_to_plot()
dates = pd.to_datetime(data['FECHA_VACUNACION'], infer_datetime_format=False, format='%Y%m%d')
generate_plot('PERÚ', data, (dates.min(), dates.max()), False, True)

In [13]:
# Departamentos
departments_ordered = np.sort(df['DEPARTAMENTO'].unique())
for d in departments_ordered:
    data = get_data_to_plot(d)
    dates = pd.to_datetime(data['FECHA_VACUNACION'], infer_datetime_format=False, format='%Y%m%d')
    generate_plot(d, data, (dates.min(), dates.max()), False, True)

<ipython-input-10-75483c5256a2>:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axs = plt.subplots(2,2, figsize=(20,20))
